# Proyecto Final: Análisis de Datos del Human Connectome Project (HCP)

En este proyecto, analizaremos datos de fMRI del Human Connectome Project (HCP) para predecir estados mentales y comportamientos usando redes neuronales. Utilizaremos PyTorch para construir y entrenar nuestros modelos.

In [1]:
# ==== Importaciones Básicas ====
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional, Union

# ==== Importaciones de PyTorch ====
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# ==== Importaciones Adicionales ====
from tqdm.notebook import tqdm

# ==== Configuración de Visualización ====
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# ==== Configuración Global ====
# Constantes para el dataset HCP
CONSTANTS = {
    'N_SUBJECTS': 100,
    'N_PARCELS': 360,
    'TR': 0.72,  # Resolución temporal en segundos
    'HEMIS': ["Right", "Left"],
    'RUNS': ['LR', 'RL'],
    'N_RUNS': 2,
}

# Definición de experimentos y condiciones
EXPERIMENTS = {
    'MOTOR': {
        'cond': ['lf', 'rf', 'lh', 'rh', 't', 'cue'],
        'description': 'Movimientos de pie izquierdo, derecho, mano izquierda, derecha, lengua y señal'
    },
    'GAMBLING': {
        'cond': ['loss', 'win'],
        'description': 'Respuestas a pérdidas y ganancias en juegos de azar'
    },
    'EMOTION': {
        'cond': ['fear', 'neut'],
        'description': 'Respuestas a expresiones de miedo y neutrales'
    },
    # ... [otros experimentos]
}

# Configuración de semilla aleatoria para reproducibilidad
def set_seed(seed: int = 42) -> None:
    """
    Configura las semillas aleatorias para reproducibilidad.
    
    Args:
        seed (int): Valor de la semilla aleatoria
    """
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
set_seed()

# Configuración de dispositivo para PyTorch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {DEVICE}")

# Configuración de rutas
HCP_DIR = "./hcp_task"
if not os.path.exists(HCP_DIR):
    os.makedirs(HCP_DIR)

# TODO: Sustituir con el setup en la libreta de proyecto


## Definición del Modelo

Definimos nuestro modelo LSTM para predecir estados mentales y comportamientos a partir de los datos de fMRI.

In [2]:
class LSTMRegression(nn.Module):
    def __init__(self, n_features: int, n_timesteps: int, n_hidden: int, n_layers: int):
        super(LSTMRegression, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.lstm = nn.LSTM(n_features, n_hidden, n_layers, batch_first=True)
        self.linear = nn.Linear(n_hidden, 1)
    
    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.n_hidden).to(DEVICE)
        c0 = torch.zeros(self.n_layers, x.size(0), self.n_hidden).to(DEVICE)
        out, _ = self.lstm(x, (h0, c0))
        out = self.linear(out[:, -1, :])
        return out

class LSTMRegression2(nn.Module):
    def __init__(self, n_features: int, n_timesteps: int, n_hidden: int, n_layers: int):
        super(LSTMRegression2, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.lstm = nn.LSTM(n_features, n_hidden, n_layers, batch_first=True)
        self.linear = nn.Linear(n_hidden, 1)
    
    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.n_hidden).to(DEVICE)
        c0 = torch.zeros(self.n_layers, x.size(0), self.n_hidden).to(DEVICE)
        out, _ = self.lstm(x, (h0, c0))
        out = self.linear(out[:, -1, :])
        return out


## Función de Entrenamiento Optimizada

Reescribimos la función de entrenamiento utilizando la versión optimizada del archivo `optimized_training.py`.

In [3]:
def load_and_prepare_data(experiment: str, regions: List[int], label_type: str) -> Tuple[np.ndarray, np.ndarray]:
    # Implementar la lógica para cargar y preparar los datos
    pass

def optimized_training(
    experiment: str,
    regions: List[int],
    model_params: Dict[str, int],
    training_params: Dict[str, Union[float, int, str]],
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
) -> Tuple[pd.DataFrame, float]:
    """
    Versión optimizada de la función de entrenamiento para modelos LSTM.
    
    Args:
        experiment (str): Nombre del experimento ('GAMBLING', 'MOTOR', etc.)
        regions (List[int]): Lista de índices de regiones cerebrales a utilizar
        model_params (Dict): Parámetros del modelo como:
            - n_features: Número de características de entrada
            - n_timesteps: Número de pasos temporales
            - n_hidden: Dimensiones del estado oculto
            - n_layers: Número de capas LSTM
        training_params (Dict): Parámetros de entrenamiento como:
            - lr: Learning rate
            - batch_size: Tamaño del batch
            - n_epochs: Número de épocas
            - label_type: Tipo de etiqueta a predecir
        device (torch.device): Dispositivo para entrenar (CPU/GPU)
    
    Returns:
        Tuple[pd.DataFrame, float]: DataFrame con métricas de entrenamiento y tiempo total
    """
    # Inicializar temporizador
    start_time = time.time()
    
    # Inicializar modelo y moverlo al dispositivo correcto
    if training_params['label_type'] == 'flanker':
        model = LSTMRegression2(**model_params).to(device)
    else:
        model = LSTMRegression(**model_params).to(device)
    
    # Inicializar optimizador y criterio
    optimizer = torch.optim.Adam(model.parameters(), lr=training_params['lr'])
    criterion = nn.MSELoss()
    
    # Inicializar scaler para precisión mixta
    scaler = GradScaler()
    
    # Cargar y preparar datos
    X, y = load_and_prepare_data(experiment, regions, training_params['label_type'])
    
    # Convertir datos a tensores y moverlos a GPU si está disponible
    X = torch.tensor(X, dtype=torch.float32).to(device)
    y = torch.tensor(y, dtype=torch.float32).to(device)
    
    # Dividir en train y test
    train_size = int(0.8 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Crear DataLoader para procesamiento por lotes eficiente
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=training_params['batch_size'],
        shuffle=True,
        pin_memory=True
    )
    
    # Listas para almacenar métricas
    metrics = []
    
    # Loop principal de entrenamiento
    for epoch in tqdm(range(training_params['n_epochs']), desc="Entrenamiento"):
        model.train()
        train_loss = 0
        
        # Training loop
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            
            # Usar precisión mixta para acelerar el entrenamiento en GPU
            with autocast():
                output = model(batch_X)
                loss = criterion(output.view(-1), batch_y)
                
                if training_params['label_type'] in ['WL', 'gender']:
                    accuracy = ((output.view(-1) > 0.5) == batch_y).float().mean()
            
            # Backward pass con scaling
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
        
        # Evaluación
        model.eval()
        with torch.no_grad():
            y_hat = model(X_test)
            test_loss = criterion(y_hat.view(-1), y_test)
            
            if training_params['label_type'] in ['WL', 'gender']:
                test_accuracy = ((y_hat.view(-1) > 0.5) == y_test).float().mean()
        
        # Guardar métricas
        metrics_dict = {
            'epoch': epoch,
            'train_loss': train_loss / len(train_loader),
            'test_loss': test_loss.item()
        }
        
        if training_params['label_type'] in ['WL', 'gender']:
            metrics_dict.update({
                'train_accuracy': accuracy.item(),
                'test_accuracy': test_accuracy.item()
            })
            
        metrics.append(metrics_dict)
    
    # Calcular tiempo total
    total_time = time.time() - start_time
    
    return pd.DataFrame(metrics), total_time

# Definir parámetros
model_params = {
    'n_features': 360,
    'n_timesteps': 67,
    'n_hidden': 20,
    'n_layers': 3
}

training_params = {
    'lr': 1e-3,
    'batch_size': 16,  # Ajustar según memoria disponible
    'n_epochs': 50,
    'label_type': 'WL'
}

# Entrenar modelo
metrics_df, training_time = optimized_training(
    experiment='GAMBLING',
    regions=list(range(360)),
    model_params=model_params,
    training_params=training_params
)

print(f"Tiempo total de entrenamiento: {training_time:.2f} segundos")
